In [73]:
import requests
import bs4
from dotenv import load_dotenv
import os
import psycopg
from datetime import datetime as dt

load_dotenv()

True

In [90]:
dbconn = os.getenv("DBCONN")
conn = psycopg.connect(dbconn)
cur = conn.cursor()

In [ ]:
cur.execute('''
    DROP TABLE IF EXISTS utoday_news;
    CREATE TABLE IF NOT EXISTS utoday_news(
        title VARCHAR(255),
        author VARCHAR(255),
        link VARCHAR(255),
        date DATE
    );
''')

In [ ]:
conn.commit()

In [ ]:
cur.close()
conn.close()

In [ ]:
response = requests.get("https://u.today/search/node?keys=bitcoin")
document = bs4.BeautifulSoup(response.text)
news_items2 = document.select("div.search-result > .news__item")

In [72]:
len(news_items), len(news_items2)

(58, 50)

In [69]:
add_to_db = []

for item in news_items:
    try: 
        data = []
        data.append(item.select_one("div.news__item-title").get_text()) # title
        data.append(item.select_one("a.humble.humble--author").get_text().split("\n")[0]) # author
        data.append(item.select_one("div.humble").get_text().split(" - ")[0]) # date
        data.append(item.select_one("a.news__item-body")["href"]) # link
        add_to_db.append(data)
    except: print("value missing")

In [ ]:
for item in add_to_db:
    item[2] = dt.strptime(item[2], "%b %d, %Y")
    cur.execute('''
        INSERT INTO utoday_news (title, author, date, link)
        VALUES (%s, %s, %s, %s);
    ''', item)

print("finished")

In [ ]:
cur.execute('''
    ALTER TABLE utoday_news
    ADD sentiment VARCHAR(255);
''')

In [ ]:
titles = cur.execute('''
    SELECT title FROM utoday_news ORDER BY date DESC;
''').fetchall()

titles = [title[0] for title in titles]

In [ ]:
hf_api_key = os.getenv("HF_API_KEY")
url = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

payload = { "inputs": titles }
headers = { "Authorization": f"Bearer {hf_api_key}" }
response = requests.post(url, headers=headers, json=payload)
print("first response", response)

if response.status_code == 503:
    headers["x-wait-for-model"] = "true"
    response = requests.post(url, headers=headers, json=payload)
    print("second response", response)

In [ ]:
import json

sentiment_results = response.json()
print(json.dumps(sentiment_results, indent=2))

In [ ]:
if len(sentiment_results) == len(titles):
    for i in range(0, len(titles)):
        label = sentiment_results[i][0]["label"]
        cur.execute('''
            UPDATE utoday_news
            SET sentiment = %s
            WHERE title = %s;
        ''', (label, titles[i]))

conn.commit()
print("finished")

In [96]:
cur.execute('''
    SELECT * FROM utoday_news ORDER BY date LIMIT 1;
''')
cur.fetchall()

[("Fidelity Says It's 'Possible' for Bitcoin to Overtake Gold",
  'Alex Dovbnya',
  'https://u.today/fidelity-says-its-possible-for-bitcoin-to-overtake-gold',
  datetime.date(2025, 3, 28),
  'positive')]

In [95]:
cur.execute('''
    SELECT * FROM utoday_news;
''')
cur.fetchall()

[("Better Than Bitcoin? 'Rich Dad Poor Dad' Author Names His Pick",
  'Alex Dovbnya',
  'https://u.today/better-than-bitcoin-rich-dad-poor-dad-author-names-his-pick',
  datetime.date(2025, 4, 2),
  'neutral'),
 ('$94 Million Bitcoin (BTC) Mystery Stuns Major US Exchange',
  'Gamza Khanzadaev',
  'https://u.today/94-million-bitcoin-btc-mystery-stuns-major-us-exchange',
  datetime.date(2025, 4, 2),
  'neutral'),
 ('Bitcoin Mogul Successfully Launched into Space, 40,000,000 XRP Withdrawal Stuns Top South Korean Exchange, Grayscale Files for Digital Large Cap Fund ETF With SEC: Crypto News Digest by U.Today',
  'Valeria Blokhina',
  'https://u.today/bitcoin-mogul-successfully-launched-into-space-40000000-xrp-withdrawal-stuns-top-south-korean',
  datetime.date(2025, 4, 2),
  'positive'),
 ("Biggest Bitcoin Critic Peter Schiff Reveals April Fools' Day Prank",
  'Gamza Khanzadaev',
  'https://u.today/biggest-bitcoin-critic-peter-schiff-reveals-april-fools-day-prank',
  datetime.date(2025, 4, 

In [ ]:
cur.execute("ROLLBACK")

In [92]:
cur.execute('''
    DELETE FROM utoday_news WHERE author = '';
''')
conn.commit()